### Predict items to buy

Need to use (user, sku) pairs for the prediction

Use simple analysis to look at the probability


In [1]:
import base
import numpy as np
import pandas as pd

pus = base.UserTrainSet(product_cate=8)

In [2]:
data = pd.DataFrame(pus.get_user_action('favor', pus.TYPE_FAVOR, 0, 6e6))
data = data.join(pus.get_user_action('buy1', pus.TYPE_BUY, 0, 3e6), how='outer')
data = data.join(pus.get_user_action('buy2', pus.TYPE_BUY, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_action('buy3', pus.TYPE_BUY, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_action('view1', pus.TYPE_VIEW, 0, 3e6), how='outer')
data = data.join(pus.get_user_action('view2', pus.TYPE_VIEW, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_action('view3', pus.TYPE_VIEW, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_cart('cart1', 0, 3e6, True), how='outer')
data = data.join(pus.get_user_cart('cart2', 0, 6e6, True), how='outer')
data = data.join(pus.get_user_action('click1', pus.TYPE_CLICK, 0, 3e6), how='outer')
data = data.join(pus.get_user_action('click2', pus.TYPE_CLICK, 3e6, 5e6), how='outer')
data = data.join(pus.get_user_action('click3', pus.TYPE_CLICK, 5e6, 6e6), how='outer')
data = data.join(pus.get_user_action('final', pus.TYPE_BUY, 6e6, 6e6 + 5 * 24 *3600), how='outer')
data[pd.isnull(data)] = 0

In [3]:
def factor_analyse(col1, col2, dt=data, print_out=True):
    f1 = dt[col1] > 0
    f2 = dt[col2] > 0
    num1 = np.sum(np.logical_and(f1, f2))
    num2 = np.sum(np.logical_and(f1, np.logical_not(f2)))
    num3 = np.sum(np.logical_and(np.logical_not(f1), f2))
    num4 = np.sum(np.logical_not(np.logical_or(f1, f2)))
    if print_out:
        print '\t{}\t{}'.format(col2, '-')
        print '{}\t{}\t{}'.format(col1, num1, num2)
        print '{}\t{}\t{}'.format('-', num3, num4)
    return num1, num2, num3, num4

In [4]:
buy_stat = pd.DataFrame()
for col in data.columns:
    buy_stat[col] = factor_analyse(col, 'final')

import pickle
with open('data/probability', 'w') as f:
    pickle.dump(buy_stat, f)

	final	-
favor	173	15256
-	1136	86476
	final	-
buy1	0	3104
-	1309	98628
	final	-
buy2	0	5430
-	1309	96302
	final	-
buy3	0	2816
-	1309	98916
	final	-
view1	289	62560
-	1020	39172
	final	-
view2	489	80301
-	820	21431
	final	-
view3	802	54685
-	507	47047
	final	-
cart1	82	15359
-	1227	86373
	final	-
cart2	532	38611
-	777	63121
	final	-
click1	277	60312
-	1032	41420
	final	-
click2	470	77066
-	839	24666
	final	-
click3	764	52300
-	545	49432
	final	-
final	1309	0
-	0	101732


In [5]:
for col in buy_stat.columns:
    prob1 = buy_stat[col][0] * 1.0 / (buy_stat[col][0] + buy_stat[col][1])
    prob2 = buy_stat[col][2] * 1.0 / (buy_stat[col][2] + buy_stat[col][3])
    if prob2 == 0.0:
        prob2 += 1e-8
    print col, '\t+\t', prob1, '\t-\t', prob2, '\tratio\t', prob1 / prob2

favor 	+	0.0112126515004 	-	0.0129662603296 	ratio	0.864756006386
buy1 	+	0.0 	-	0.0130982518987 	ratio	0.0
buy2 	+	0.0 	-	0.0134103738308 	ratio	0.0
buy3 	+	0.0 	-	0.0130606136194 	ratio	0.0
view1 	+	0.00459832296457 	-	0.0253781847134 	ratio	0.181191957443
view2 	+	0.00605272929818 	-	0.0368522763022 	ratio	0.164243023919
view3 	+	0.0144538360337 	-	0.010661563696 	ratio	1.35569569772
cart1 	+	0.00531053688233 	-	0.0140068493151 	ratio	0.379138574484
cart2 	+	0.013591191273 	-	0.012160005008 	ratio	1.11769619043
click1 	+	0.00457178695803 	-	0.0243098087251 	ratio	0.188063468936
click2 	+	0.00606170037144 	-	0.0328955106842 	ratio	0.184271356345
click3 	+	0.0143977084276 	-	0.0109050163075 	ratio	1.32028307171
final 	+	1.0 	-	1e-08 	ratio	100000000.0


In [6]:
# More complex interactions
for col in data.columns:
    if col != 'final':
        sub_set = data[data[col] > 0]
        print col, "+ subset"

        for col2 in sub_set.columns:
            if col2 != col and col2 != 'final':
                ret = factor_analyse(col2, 'final', sub_set, False)
                prob1 = ret[0] * 1.0 / (ret[0] + ret[1])
                prob2 = ret[2] * 1.0 / (ret[2] + ret[3])
                if ret[2] == 0:
                    prob2 = 1.0 / (ret[2] + ret[3])
                print col2, '\t', prob1, '\t', prob2, '\t', prob1 / prob2
    print

favor + subset
buy1 	0.0 	0.0116091799758 	0.0
buy2 	0.0 	0.0120515499826 	0.0
buy3 	0.0 	0.0116271254789 	0.0
view1 	0.00611741489887 	0.0209671326029 	0.291762112384
view2 	0.00763417054737 	0.031330472103 	0.243665991443
view3 	0.016330451489 	0.003300330033 	4.94812680115
cart1 	0.00397877984085 	0.0129702731008 	0.306761454438
cart2 	0.012815483196 	0.00963762528913 	1.32973453642
click1 	0.00611184679637 	0.0201354240912 	0.303537028507
click2 	0.00769956002514 	0.0277674935209 	0.277286821705
click3 	0.0166519629466 	0.00345857569565 	4.81468801379

buy1 + subset
favor 	0.0 	0.000388048117967 	0.0
buy2 	0.0 	0.000322268772156 	0.0
buy3 	nan 	0.000322164948454 	nan
view1 	0.0 	0.2 	0.0
view2 	0.0 	0.000630914826498 	0.0
view3 	0.0 	0.000396196513471 	0.0
cart1 	0.0 	0.00178571428571 	0.0
cart2 	0.0 	0.0017667844523 	0.0
click1 	0.0 	0.00961538461538 	0.0
click2 	0.0 	0.000607164541591 	0.0
click3 	0.0 	0.000392156862745 	0.0

buy2 + subset
favor 	0.0 	0.000229568411387 	0.0
buy1 

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


buy1 	0.0 	0.00616871239167 	0.0
buy2 	0.0 	0.0064870458073 	0.0
buy3 	0.0 	0.00617330707469 	0.0
view1 	0.00387644597588 	0.00936504963476 	0.413926901304
view3 	0.0075613929612 	0.00431493714042 	1.75237615639
cart1 	0.00459418070444 	0.00630106031636 	0.729112319797
cart2 	0.00861361771944 	0.00439954375102 	1.95784340534
click1 	0.0039595529537 	0.00896052047908 	0.441888723199
click2 	0.00604260813428 	0.0062874251497 	0.961062436595
click3 	0.00778605280975 	0.00423492417711 	1.83853417066

view3 + subset
favor 	0.016330451489 	0.0140725963832 	1.16044339255
buy1 	0.0 	0.0146065164733 	0.0
buy2 	0.0 	0.0149518074535 	0.0
buy3 	0.0 	0.015222260183 	0.0
view1 	0.00668802140167 	0.022601373809 	0.295912162605
view2 	0.0075613929612 	0.0388040192795 	0.194861076291
cart1 	0.00908190224571 	0.0151119742672 	0.600973908845
cart2 	0.0196860415783 	0.0105899677288 	1.85893310371
click1 	0.0066261531703 	0.0220439459018 	0.300588342932
click2 	0.00771177250271 	0.0344482561464 	0.22386539

#### Summary

Favor will increase probability for buying.

Recent click/viewing/adding to cart increases probability.

If the item is added to cart/viewed/clicked early on, it's less likely to be bought.

If the item has been bought, it will never be bought again.


In [7]:
train_data = data.drop('final', 1).as_matrix()
train_label = np.array(data['final'] > 0, dtype=int)
mean_dt = np.mean(train_data, axis=0)
std_dt = np.std(train_data, axis=0)
train_data = (train_data - mean_dt) / std_dt

In [8]:
nrow = len(train_data)
test_size = int(nrow * 0.3)
import random
idx = random.sample(xrange(nrow), test_size)
mask = np.ones(nrow, dtype=bool)
mask[idx] = False
np.save('train_data4', train_data[mask, :])
np.save('train_label4', train_label[mask])
np.save('test_data4', train_data[idx, :])
np.save('test_label4', train_label[idx])